<a href="https://colab.research.google.com/github/schenita/info5731/blob/master/Floyd_SelfStudy10_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Schenita Floyd
# Self Study 10 Part 2 - Grus: Chapter 11, 13, 17

In [0]:
# Chapter 11 - Machine Learning

In [0]:
#Overfitting and Underfitting

In [0]:
def split_data(data, prob):
  """split data into fractions [prob, 1 - prob]""" 
  results = [], []
  for row in data:
    results[0 if random.random() < prob else 1].append(row) 
  return results

In [0]:
def train_test_split(x, y, test_pct):
  data = zip(x, y)
  train, test = split_data(data, 1 - test_pct) 
  x_train, y_train = zip(*train)
  x_test, y_test = zip(*test)
  return x_train, x_test, y_train, y_test

In [0]:
import math
model = mean()
x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.33)
model.train(x_train, y_train)
performance = model.test(x_test, y_test)

In [0]:
def accuracy(tp, fp, fn, tn): 
  correct = tp + tn 
  total = tp + fp + fn + tn 
  return correct / total

In [11]:
print accuracy(70, 4930, 13930, 981070)

0


In [0]:
def precision(tp, fp, fn, tn):
  return tp / (tp + fp)

In [13]:
print precision(70, 4930, 13930, 981070)

0


In [14]:
def recall(tp, fp, fn, tn): 
  return tp / (tp + fn)
print recall(70, 4930, 13930, 981070)

0


In [0]:
def f1_score(tp, fp, fn, tn):
    p = precision(tp, fp, fn, tn) 
    r = recall(tp, fp, fn, tn)

    return 2 * p * r / (p+r)

In [0]:
# Chapter 13 - Naive Bayes

In [0]:
def tokenize(message):
    message = message.lower()
    all_words = re.findall("[a-z0-9']+", message) 
    return set(all_words)

In [0]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)""" 
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message): 
          counts[word][0 if is_spam else 1] += 1
    return counts

In [0]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5): 
  """turn the word_counts into a list of triplets
  w, p(w | spam) and p(w | ~spam)"""
  return [(w,
          (spam + k) / (total_spams + 2 * k),
          (non_spam + k) / (total_non_spams + 2 * k)) 
          for w, (spam, non_spam) in counts.iteritems()]

In [0]:
def spam_probability(word_probs, message): 
  message_words = tokenize(message) 
  log_prob_if_spam = log_prob_if_not_spam = 0.0
  for word, prob_if_spam, prob_if_not_spam in word_probs:
    if word in message_words:
      log_prob_if_spam += math.log(prob_if_spam)
      log_prob_if_not_spam += math.log(prob_if_not_spam)

    else:
      log_prob_if_spam += math.log(1.0 - prob_if_spam)
      log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

  prob_if_spam = math.exp(log_prob_if_spam) 
  prob_if_not_spam = math.exp(log_prob_if_not_spam) 
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [0]:
class NaiveBayesClassifier:
  def __init__(self, k=0.5): 
    self.k = k
    self.word_probs = []
    
  def train(self, training_set):
            # count spam and non-spam messages
    num_spams = len([is_spam
                    for message, is_spam in training_set
                    if is_spam])
    num_non_spams = len(training_set) - num_spams
            # run training data through our "pipeline"
    word_counts = count_words(training_set)
     self.word_probs = word_probabilities(word_counts,
                                          num_spams,
                                          num_non_spams,
                                          self.k)
  def classify(self, message):
    return spam_probability(self.word_probs, message)


In [0]:
import glob, re

    path = r"C:\spam\*\*"
    data = []
    # glob.glob returns every filename that matches the wildcarded path
    for fn in glob.glob(path): 
      is_spam = "ham" not in fn
      with open(fn,'r') as file: 
        for line in file:
        if line.startswith("Subject:"):
            # remove the leading "Subject: " and keep what's left 
            subject = re.sub(r"^Subject: ", "", line).strip() 
            data.append((subject, is_spam))
    random.seed(0) # just so you get the same answers as me 
    train_data, test_data = split_data(data, 0.75)



In [0]:
classifier = NaiveBayesClassifier()
classifier.train(train_data)

In [0]:
classified = [(subject, is_spam, classifier.classify(subject)) 
              for subject, is_spam in test_data]


In [0]:
counts = Counter((is_spam, spam_probability > 0.5)
              for _, is_spam, spam_probability in classified)

In [0]:
classified.sort(key=lambda row: row[2])

In [0]:
spammiest_hams = filter(lambda row: not row[1], classified)[-5:]

In [0]:
hammiest_spams = filter(lambda row: row[1], classified)[:5]

In [0]:
def p_spam_given_word(word_prob):
        """uses bayes's theorem to compute p(spam | message contains word)"""
        # word_prob is one of the triplets produced by word_probabilities
        word, prob_if_spam, prob_if_not_spam = word_prob 
        return prob_if_spam / (prob_if_spam + prob_if_not_spam)
words = sorted(classifier.word_probs, key=p_spam_given_word)

In [0]:
spammiest_words = words[-5:]
hammiest_words = words[:5]

In [0]:
def drop_final_s(word):
  return re.sub("s$", "", word)

In [0]:
#Chapter 17 - Decision Trees

In [0]:
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy""" 
    return sum(-p * math.log(p, 2)
              for p in class_probabilities
              if p)

In [0]:
def class_probabilities(labels): 
  total_count = len(labels) 
  return [count / total_count
          for count in Counter(labels).values()]

In [0]:
def data_entropy(labeled_data):
  labels = [label for _, label in labeled_data] 
  probabilities = class_probabilities(labels) 
  return entropy(probabilities)

In [0]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets subsets is a list of lists of labeled data"""
    total_count = sum(len(subset) for subset in subsets)
    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

In [0]:
inputs = [
        ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False), 
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'}, False), 
        ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True), 
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'}, True),
        ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'},  True),
        ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
        ({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'},  True),
        ({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'},  False),
        ({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'},       True),
        ({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'},  True),
        ({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
        ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'},     True),
        ({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'},       True),
        ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)]


In [0]:
def partition_by(inputs, attribute):
    """each input is a pair (attribute_dict, label) 
    returns a dict : attribute_value -> inputs""" 
    groups = defaultdict(list)
    for input in inputs:
            key = input[0][attribute]
            groups[key].append(input) 
    return groups

In [0]:
def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition""" 
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [0]:
for key in ['level','lang','tweets','phd']: 
  print key, partition_entropy_by(inputs, key)


In [47]:
senior_inputs = [(input, label)
                for input, label in inputs if input["level"] == "Senior"]

In [0]:
for key in ['lang', 'tweets', 'phd']:
    print key, partition_entropy_by(senior_inputs, key)

In [49]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    # if this is a leaf node, return its value
    if tree in [True, False]: 
      return tree
    attribute, subtree_dict = tree
    subtree_key = input.get(attribute) 
    if subtree_key not in subtree_dict:
      subtree_key = None
    subtree = subtree_dict[subtree_key]
    return classify(subtree, input)

In [0]:
def build_tree_id3(inputs, split_candidates=None):
  if split_candidates is None: 
    split_candidates = inputs[0][0].keys()
    # count Trues and Falses in the inputs
  num_inputs = len(inputs)
  num_trues = len([label for item, label in inputs if label]) 
  num_falses = num_inputs - num_trues
  
  if num_trues == 0: return False
  if num_falses == 0: return True
  
  if not split_candidates:
    return num_trues >= num_falses
  best_attribute = min(split_candidates,
                             key=partial(partition_entropy_by, inputs))
  partitions = partition_by(inputs, best_attribute) 
  new_candidates = [a for a in split_candidates
                    if a != best_attribute]
  subtrees = { attribute_value : build_tree_id3(subset, new_candidates) 
              for attribute_value, subset in partitions.iteritems() }
  subtrees[None] = num_trues > num_falses  
  return (best_attribute, subtrees)
    

In [0]:
tree = build_tree_id3(inputs)
classify(tree, { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "no"} )

In [0]:
classify(tree, { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} )

In [0]:
#random forest

In [0]:
def forest_classify(trees, input):
  votes = [classify(tree, input) for tree in trees] 
  vote_counts = Counter(votes)
  return vote_counts.most_common(1)[0][0]

In [0]:
# if there's already few enough split candidates, look at all of them
if len(split_candidates) <= self.num_split_candidates: 
  sampled_split_candidates = split_candidates
        # otherwise pick a random sample
else:
  sampled_split_candidates = random.sample(split_candidates,
                                                     self.num_split_candidates)
# now choose the best attribute only from those candidates
best_attribute = min(sampled_split_candidates,
                     key=partial(partition_entropy_by, inputs))
partitions = partition_by(inputs, best_attribute)